## Instalação Bibliotecas

In [1]:
%pip install oracledb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.9 MB/s eta 0:00:00


In [2]:
!pip install cx_Oracle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 13.2 MB/s eta 0:00:00


## Bibliotecas

In [3]:
import oracledb
import cx_Oracle
import getpass
import pandas as pd
import random
from datetime import datetime, timedelta

##1) Gere mais 50 produtos, 20 vendedores, 100 clientes, 1000 vendas.

In [11]:
pw = getpass.getpass("Digite sua senha: ")

connection = oracledb.connect(
    user="RM553808",
    password=pw,
    dsn="oracle.fiap.com.br:1521/orcl")

print("Conexão bem-sucedida ao Oracle Database.")

cursor = connection.cursor()

Digite sua senha: ··········
Conexão bem-sucedida ao Oracle Database.


In [5]:
# inserindo 50 produtos
for i in range(11, 62, 1):
    cursor.execute(f"""
        INSERT INTO produtos (produto_id, nome, preco)
        VALUES ({i}, 'Produto Eletrônico {i}', {random.uniform(100.0, 1000.0):.2f})
    """)

# inserindo 100 clientes
for i in range(31, 132, 1):
    cursor.execute(f"""
        INSERT INTO clientes (cliente_id, nome, email)
        VALUES ({i}, 'Cliente Genérico {i}', 'cliente{i}@exemplo.com')
    """)

# inserindo 20 vendedores
for i in range(11, 32, 1):
    cursor.execute(f"""
        INSERT INTO vendedores (vendedor_id, nome)
        VALUES ({i}, 'Vendedor {i}')
    """)

# inserindo 1000 vendas
for i in range(201, 1202, 1):
    data_venda = datetime.now() - timedelta(days=random.randint(1, 365))
    cursor.execute(f"""
        INSERT INTO vendas (venda_id, cliente_id, produto_id, vendedor_id, quantidade, data_venda)
        VALUES ({i}, {random.randint(31, 131)}, {random.randint(11, 61)}, {random.randint(11, 31)}, {random.randint(1, 5)}, TO_DATE('{data_venda.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'))
    """)

connection.commit()
print("Dados inseridos com sucesso.")

Dados inseridos com sucesso.


##2) Faça uma consulta  por Cliente, Produto, Vendedor e Data. Quando qualquer destes itens for branco (sem digitação) o programa deve resolver de duas formas em células difierentes. Uma solução é condicional dentro do próprio SQL e outra é resolvida no Python.

In [6]:
cliente_nome = input("Digite o nome do cliente (ou deixe em branco): ")
produto_nome = input("Digite o nome do produto (ou deixe em branco): ")
vendedor_nome = input("Digite o nome do vendedor (ou deixe em branco): ")
data_venda = input("Digite a data da venda (ou deixe em branco): ")

try:
    # Consulta SQL com condições dinâmicas
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        WHERE (:cliente_nome IS NULL OR c.nome LIKE :cliente_nome)
        AND (:produto_nome IS NULL OR p.nome LIKE :produto_nome)
        AND (:vendedor_nome IS NULL OR vd.nome LIKE :vendedor_nome)
        AND (:data_venda IS NULL OR v.data_venda = TO_DATE(:data_venda, 'YYYY-MM-DD'))
        ORDER BY v.data_venda DESC
    """, {'cliente_nome': cliente_nome or None, 'produto_nome': produto_nome or None,
          'vendedor_nome': vendedor_nome or None, 'data_venda': data_venda or None})

    results = cursor.fetchall()

    if results:
        df = pd.DataFrame(results, columns=['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade', 'Valor Total Venda', 'Nome Vendedor', 'Data Venda'])
        print("Consulta com condições dinâmicas no SQL:")
        print(df)
    else:
        print("Nenhuma compra encontrada para os critérios informados.")

except cx_Oracle.DatabaseError as e:
    print("Erro ao buscar compras:", e)




Digite o nome do cliente (ou deixe em branco): 
Digite o nome do produto (ou deixe em branco): 
Digite o nome do vendedor (ou deixe em branco): 
Digite a data da venda (ou deixe em branco): 2024-03-09
Consulta com condições dinâmicas no SQL:
   ID Venda         Nome Cliente           Nome Produto   Preço  Quantidade  \
0      1086  Cliente Genérico 37  Produto Eletrônico 40  763.13           5   
1       959  Cliente Genérico 62  Produto Eletrônico 20  445.29           4   

   Valor Total Venda Nome Vendedor Data Venda  
0            3815.65   Vendedor 23 2024-03-09  
1            1781.16   Vendedor 15 2024-03-09  


- CONSULTA EM PYTHON

In [12]:
try:
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        ORDER BY v.data_venda DESC
    """)

    results = cursor.fetchall()

    if results:
        df = pd.DataFrame(results, columns=['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade', 'Valor Total Venda', 'Nome Vendedor', 'Data Venda'])
        # Filtrando os resultados em Python conforme as entradas do usuário
        if cliente_nome:
            df = df[df['Nome Cliente'].str.contains(cliente_nome, case=False)]
        if produto_nome:
            df = df[df['Nome Produto'].str.contains(produto_nome, case=False)]
        if vendedor_nome:
            df = df[df['Nome Vendedor'].str.contains(vendedor_nome, case=False)]
        if data_venda:
            df = df[df['Data Venda'] == pd.to_datetime(data_venda)]

        if not df.empty:
            print("\nConsulta filtrada em Python:")
            print(df)
        else:
            print("\nNenhuma compra encontrada para os critérios informados.")
    else:
        print("\nNenhuma compra encontrada.")

except cx_Oracle.DatabaseError as e:
    print("\nErro ao buscar compras:", e)



Consulta filtrada em Python:
     ID Venda         Nome Cliente           Nome Produto   Preço  Quantidade  \
622       959  Cliente Genérico 62  Produto Eletrônico 20  445.29           4   
623      1086  Cliente Genérico 37  Produto Eletrônico 40  763.13           5   

     Valor Total Venda Nome Vendedor Data Venda  
622            1781.16   Vendedor 15 2024-03-09  
623            3815.65   Vendedor 23 2024-03-09  


In [13]:
df.head(5)


,ID Venda,Nome Cliente,Nome Produto,Preço,Quantidade,Valor Total Venda,Nome Vendedor,Data Venda
622,959,Cliente Genérico 62,Produto Eletrônico 20,445.29,4,1781.16,Vendedor 15,2024-03-09
623,1086,Cliente Genérico 37,Produto Eletrônico 40,763.13,5,3815.65,Vendedor 23,2024-03-09


##3) Calcule a mediana por total de vendas por cliente via python mas selecionando por SQL

In [15]:
cursor.execute("""
    SELECT c.nome AS cliente_nome, SUM(p.preco * v.quantidade) AS valor_total_venda
    FROM vendas v
    JOIN clientes c ON v.cliente_id = c.cliente_id
    JOIN produtos p ON v.produto_id = p.produto_id
    GROUP BY c.nome
""")
results = cursor.fetchall()

# Extraindo os valores do total de vendas para uma lista, pois o results nao é uma lista, apenas guarda o resultado do select
valor_total_vendas = [result[1] for result in results]

mediana = pd.Series(valor_total_vendas).median()
print(f"Mediana do total de vendas por cliente: {mediana:.2f}")
#fechando o cursor e a conexão
cursor.close()
connection.close()

Mediana do total de vendas por cliente: 15786.91
